# Setup

In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Import d. Daten

In [23]:
df = pd.read_csv("../../results/subset_50h/features_pca_vif.csv")
pd.set_option('display.max_columns', None)
df.head()

,rms,log_energy,clipping_ratio,crest_factor,snr,f0,phoneme_entropy,rt60_reverberation,spectral_bandwidth,spectral_flatness,vad,zcr,loudness_range,chroma_1,chroma_2,chroma_3,chroma_4,chroma_6,chroma_7,chroma_9,chroma_10,chroma_11,pca_mfcc_1,pca_mfcc_2,pca_mfcc_3,pca_mfcc_4,pca_mfcc_5,pca_mfcc_6,pca_mfcc_7,pca_mfcc_8,pca_mfcc_9,pca_mfcc_10,wer
0,0.138755,7.460141,0.000044,7.206935,32.660785,100.887721,5.278837,4.124500,1566.968846,0.016414,53.107345,0.155728,55.996592,0.374366,0.391721,0.378379,0.402801,0.485978,0.521542,0.524347,0.532465,0.526100,1.854109,-2.398333,3.225753,-0.039237,1.634311,-0.488640,1.029713,-1.378703,-0.388375,-0.740754,0.142857
1,0.107539,7.020273,0.000052,9.298929,30.196857,135.333079,5.394557,5.490875,1781.131421,0.037886,52.631579,0.146282,46.252195,0.367287,0.367356,0.352975,0.383940,0.473410,0.517349,0.483220,0.377662,0.322873,-2.436086,-1.492873,-4.327856,-1.205561,0.111216,2.443767,4.131992,-1.766842,0.937929,1.570614,0.000000
2,0.111006,7.192654,0.000065,9.008530,21.029661,182.299178,5.082705,6.386625,1835.400159,0.028636,45.497630,0.153162,47.596651,0.142349,0.127142,0.216322,0.356451,0.295204,0.289746,0.448400,0.385253,0.370323,2.021014,1.720223,0.441823,-0.357880,-2.193234,-0.315723,4.903598,-0.055908,1.331785,0.655732,0.111111
3,0.100967,7.010160,0.000101,9.904255,20.688646,96.783083,5.229240,5.590375,1671.440418,0.044127,62.910798,0.159553,39.648168,0.483797,0.491001,0.445326,0.420861,0.468498,0.600653,0.588488,0.506522,0.428744,-3.548503,-3.068428,3.095815,-0.682608,0.278585,1.135436,0.206216,-1.896255,1.646602,1.353080,0.333333
4,0.160976,7.385242,0.000209,6.212097,35.451932,102.276770,5.257475,2.706875,1423.227017,0.022996,61.475410,0.146696,80.180676,0.401988,0.271837,0.262677,0.196845,0.266492,0.350469,0.449861,0.480996,0.442719,-0.236775,-2.975602,-2.736656,-1.410569,0.626282,1.054996,0.698175,-0.081188,-3.044000,0.450005,0.454545


# Ziel- und Eingabefeatures

In [15]:
# Zielvariable
target_column = "wer"

# Features
feature_columns = df.select_dtypes(include='number').columns.drop(target_column)

# X und y
X = df[feature_columns]
y = df[target_column]

# Train-Test-Split & Skalieren

In [16]:
# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Skalieren
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# PyCaret

In [7]:
from pycaret.regression import *

# Setup konfigurieren
reg_setup = setup(
    data=df,
    target="wer",
    session_id=42,
    normalize=True,
    verbose=True
)

# Automatischer Modellvergleich
best_model = compare_models()

# Modell erstellen (z. B. das automatisch gewählte)
final_model = finalize_model(best_model)

# Modell evaluieren (verschiedene Plots möglich)
evaluate_model(final_model)

,Description,Value
0,Session id,42
1,Target,wer
2,Target type,Regression
3,Original data shape,"(23287, 33)"
4,Transformed data shape,"(23287, 33)"
5,Transformed train set shape,"(16300, 33)"
6,Transformed test set shape,"(6987, 33)"
7,Numeric features,32
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.1989,0.0922,0.3024,0.1008,0.1971,0.6235,0.8920
catboost,CatBoost Regressor,0.1943,0.0928,0.3034,0.0945,0.1960,0.6024,0.9610
lightgbm,Light Gradient Boosting Machine,0.1949,0.0936,0.3046,0.0879,0.1970,0.5959,0.4330
gbr,Gradient Boosting Regressor,0.1970,0.0945,0.3062,0.0772,0.1971,0.5905,1.7320
br,Bayesian Ridge,0.1996,0.0951,0.3069,0.0749,0.1984,0.6136,0.0110
lr,Linear Regression,0.1996,0.0951,0.3070,0.0746,0.1985,0.6164,0.2920
ridge,Ridge Regression,0.1996,0.0951,0.3070,0.0746,0.1985,0.6164,0.0110
rf,Random Forest Regressor,0.2011,0.0958,0.3084,0.0636,0.2004,0.6367,7.8090
lar,Least Angle Regression,0.2017,0.0966,0.3095,0.0582,0.2010,0.6367,0.0130
omp,Orthogonal Matching Pursuit,0.2046,0.0998,0.3144,0.0289,0.2030,0.6172,0.0110


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

# Flaml

In [22]:
from flaml import AutoML

automl = AutoML()
automl.fit(X_train, y_train, task="regression", time_budget=60)  # 5 Min Budget

# Vorhersage und Bewertung
y_pred = automl.predict(X_test)
from sklearn.metrics import r2_score
print("R2:", r2_score(y_test, y_pred))

[flaml.automl.logger: 07-31 10:17:38] {1752} INFO - task = regression
[flaml.automl.logger: 07-31 10:17:38] {1763} INFO - Evaluation method: holdout
[flaml.automl.logger: 07-31 10:17:38] {1862} INFO - Minimizing error metric: 1-r2
[flaml.automl.logger: 07-31 10:17:38] {1979} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'sgd', 'catboost']
[flaml.automl.logger: 07-31 10:17:38] {2282} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 07-31 10:17:38] {2417} INFO - Estimated sufficient time budget=267s. Estimated necessary time budget=2s.
[flaml.automl.logger: 07-31 10:17:38] {2466} INFO -  at 0.1s,	estimator lgbm's best error=0.9832,	best estimator lgbm's best error=0.9832
[flaml.automl.logger: 07-31 10:17:38] {2282} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 07-31 10:17:38] {2466} INFO -  at 0.2s,	estimator lgbm's best error=0.9832,	best estimator lgbm's best error=0.9832
[flaml.automl.logger: 07-31 10